In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import numbers
from openpyxl.utils import get_column_letter

file_path = r"C:\Users\brend\OneDrive - Stonehill College\All Cape League Trackman.xlsx"
df = pd.read_excel(file_path)

In [2]:
df = df[df["PitcherTeam"] == "FAL_COM"]

df = df[
    df["TaggedPitchType"].notna() &
    df["Balls"].notna() &
    df["Strikes"].notna() &
    df["PlayResult"].notna()
].copy()

df["Count"] = df["Balls"].astype(str) + "-" + df["Strikes"].astype(str)

df["PA"] = (
    df["PlayResult"].isin(["Single", "Double", "Triple", "HomeRun", "Out", "Error", "FieldersChoice", "Sacrifice"]) |
    df["KorBB"].isin(["Strikeout", "Walk"]) |
    df["PitchCall"].eq("HitByPitch")
).astype(int)
df["AB"] = (
    df["PlayResult"].isin(["Single", "Double", "Triple", "HomeRun", "Out", "Error", "FieldersChoice"]) |
    df["KorBB"].eq("Strikeout")
).astype(int)
df["H"] = df["PlayResult"].isin(["Single", "Double", "Triple", "HomeRun"]).astype(int)
df["1B"] = (df["PlayResult"] == "Single").astype(int)
df["2B"] = (df["PlayResult"] == "Double").astype(int)
df["3B"] = (df["PlayResult"] == "Triple").astype(int)
df["HR"] = (df["PlayResult"] == "HomeRun").astype(int)
df["BB"] = (df["KorBB"] == "Walk").astype(int)
df["HBP"] = (df["PitchCall"] == "HitByPitch").astype(int)
df["SF"] = (
    (df["PlayResult"] == "Sacrifice") &
    (df["TaggedHitType"].isin(["LineDrive", "FlyBall"]))
).astype(int)

df["PA_ID"] = (
    df["Date"].astype(str) + "_" +
    df["Pitcher"].astype(str) + "_" +
    df["Batter"].astype(str) + "_" +
    df["Inning"].astype(str) + "_" +
    df["PAofInning"].astype(str)
)

last_pitch_df = df.sort_values("PitchNo").groupby("PA_ID").tail(1)

count_summary = df.groupby("Count").agg(
    PA=("PA", "sum"),
    AB=("AB", "sum"),
    H=("H", "sum"),
    BB=("BB", "sum"),
    HBP=("HBP", "sum"),
    SF=("SF", "sum"),
    _1B=("1B", "sum"),
    _2B=("2B", "sum"),
    _3B=("3B", "sum"),
    HR=("HR", "sum")
).reset_index()

count_summary["AVG"] = count_summary["H"] / count_summary["AB"]
count_summary["OBP"] = (count_summary["H"] + count_summary["BB"] + count_summary["HBP"]) / (
    count_summary["AB"] + count_summary["BB"] + count_summary["HBP"] + count_summary["SF"]
)
count_summary["TB"] = (
    count_summary["_1B"]
    + 2 * count_summary["_2B"]
    + 3 * count_summary["_3B"]
    + 4 * count_summary["HR"]
)
count_summary["SLG"] = count_summary["TB"] / count_summary["AB"]
count_summary["OPS"] = count_summary["OBP"] + count_summary["SLG"]

count_summary[["AVG", "OBP", "SLG", "OPS"]] = count_summary[["AVG", "OBP", "SLG", "OPS"]].round(3)

count_summary.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
count_summary.dropna(subset=["AVG", "OBP", "SLG"], inplace=True)

count_summary = count_summary.sort_values(by="OPS", ascending=False)

best_counts = count_summary.head(3)["Count"].tolist()

print("\n TOP 3 COUNTS BY HIGHEST OPS Allowed:")
print(count_summary.head(3)[["Count", "PA", "AB", "AVG", "OBP", "SLG", "OPS"]])

for count in best_counts:
    print(f"\n PITCH TYPES BY WORST COUNT: {count}")
    
    df_wc = last_pitch_df[last_pitch_df["Count"] == count].copy()
    
    pitch_summary = df_wc.groupby("TaggedPitchType").agg(
        PA=("PlayResult", "count"),
        AB=("AB", "sum"),
        H=("H", "sum"),
        BB=("BB", "sum"),
        HBP=("HBP", "sum"),
        SF=("SF", "sum"),
        _1B=("1B", "sum"),
        _2B=("2B", "sum"),
        _3B=("3B", "sum"),
        HR=("HR", "sum")
    ).reset_index()
    
    total_pa = pitch_summary["PA"].sum()
    pitch_summary["Usage%"] = (pitch_summary["PA"] / total_pa) * 100
    
    pitch_summary["AVG"] = pitch_summary["H"] / pitch_summary["AB"]
    pitch_summary["OBP"] = (pitch_summary["H"] + pitch_summary["BB"] + pitch_summary["HBP"]) / (
        pitch_summary["AB"] + pitch_summary["BB"] + pitch_summary["HBP"] + pitch_summary["SF"]
    )
    pitch_summary["TB"] = (
        pitch_summary["_1B"]
        + 2 * pitch_summary["_2B"]
        + 3 * pitch_summary["_3B"]
        + 4 * pitch_summary["HR"]
    )
    pitch_summary["SLG"] = pitch_summary["TB"] / pitch_summary["AB"]
    pitch_summary["OPS"] = pitch_summary["OBP"] + pitch_summary["SLG"]
    
    pitch_summary.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    pitch_summary.dropna(subset=["AVG", "OBP", "SLG"], inplace=True)

    total_pa = pitch_summary["PA"].sum()
    pitch_summary["Usage%"] = (pitch_summary["PA"] / total_pa) * 100

    pitch_summary[["AVG", "OBP", "SLG", "OPS"]] = pitch_summary[["AVG", "OBP", "SLG", "OPS"]].round(3)
    pitch_summary["Usage%"] = pitch_summary["Usage%"].round(2)

    pitch_summary = pitch_summary.sort_values(by="OPS", ascending=False)
    
    print(pitch_summary[["TaggedPitchType", "Usage%", "AVG", "OBP", "SLG", "OPS"]].round({
        "Usage%": 2, "AVG": 3, "OBP": 3, "SLG": 3, "OPS": 3
    }))

report_rows = []

report_rows.append(["TOP 3 COUNTS BY HIGHEST OPS Allowed:"])
report_rows.append(["Count", "PA", "AB", "AVG", "OBP", "SLG", "OPS"])

for _, row in count_summary.head(3).iterrows():
    report_rows.append([
        row["Count"], row["PA"], row["AB"],
        round(row["AVG"], 3), round(row["OBP"], 3),
        round(row["SLG"], 3), round(row["OPS"], 3)
    ])

for count in best_counts:
    report_rows.append([""])
    report_rows.append([f"PITCH TYPES BY WORST COUNT: {count}"])
    report_rows.append(["TaggedPitchType", "Usage%", "AVG", "OBP", "SLG", "OPS"])

    df_wc = last_pitch_df[last_pitch_df["Count"] == count].copy()

    pitch_summary = df_wc.groupby("TaggedPitchType").agg(
        PA=("PlayResult", "count"),
        AB=("AB", "sum"),
        H=("H", "sum"),
        BB=("BB", "sum"),
        HBP=("HBP", "sum"),
        SF=("SF", "sum"),
        _1B=("1B", "sum"),
        _2B=("2B", "sum"),
        _3B=("3B", "sum"),
        HR=("HR", "sum")
    ).reset_index()

    total_pa = pitch_summary["PA"].sum()
    pitch_summary["Usage%"] = (pitch_summary["PA"] / total_pa) * 100
    pitch_summary["AVG"] = pitch_summary["H"] / pitch_summary["AB"]
    pitch_summary["OBP"] = (pitch_summary["H"] + pitch_summary["BB"] + pitch_summary["HBP"]) / (
        pitch_summary["AB"] + pitch_summary["BB"] + pitch_summary["HBP"] + pitch_summary["SF"]
    )
    pitch_summary["TB"] = (
        pitch_summary["_1B"]
        + 2 * pitch_summary["_2B"]
        + 3 * pitch_summary["_3B"]
        + 4 * pitch_summary["HR"]
    )
    pitch_summary["SLG"] = pitch_summary["TB"] / pitch_summary["AB"]
    pitch_summary["OPS"] = pitch_summary["OBP"] + pitch_summary["SLG"]
    pitch_summary.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    pitch_summary.dropna(subset=["AVG", "OBP", "SLG"], inplace=True)
    total_pa = pitch_summary["PA"].sum()
    pitch_summary["Usage%"] = (pitch_summary["PA"] / total_pa) * 100
    pitch_summary[["AVG", "OBP", "SLG", "OPS"]] = pitch_summary[["AVG", "OBP", "SLG", "OPS"]].round(3)
    pitch_summary["Usage%"] = pitch_summary["Usage%"].round(2)
    pitch_summary = pitch_summary.sort_values(by="OPS", ascending=False)

    for _, row in pitch_summary.iterrows():
        report_rows.append([
            row["TaggedPitchType"],
            row["Usage%"],
            row["AVG"],
            row["OBP"],
            row["SLG"],
            row["OPS"]
        ])

report_df = pd.DataFrame(report_rows)

output_file = "FAL_COM_Report.xlsx"
report_df.to_excel(output_file, sheet_name="Sheet1", index=False, header=False)


 TOP 3 COUNTS BY HIGHEST OPS Allowed:
   Count  PA  AB    AVG    OBP    SLG    OPS
10   3-1  78  30  0.300  0.731  0.367  1.097
9    3-0  45   2  0.000  0.956  0.000  0.956
3    1-0  98  96  0.365  0.378  0.510  0.888

 PITCH TYPES BY WORST COUNT: 3-1
  TaggedPitchType  Usage%    AVG    OBP    SLG    OPS
4          Sinker   16.44  0.500  0.833  0.500  1.333
2        Fastball   35.62  0.455  0.769  0.545  1.315
1          Cutter    8.22  0.000  0.833  0.000  0.833
3       Four-Seam   32.88  0.167  0.583  0.250  0.833
6        Two-Seam    6.85  0.000  0.600  0.000  0.600

 PITCH TYPES BY WORST COUNT: 3-0
  TaggedPitchType  Usage%  AVG    OBP  SLG    OPS
3       Four-Seam   94.44  0.0  0.941  0.0  0.941
0       Curveball    5.56  0.0  0.000  0.0  0.000

 PITCH TYPES BY WORST COUNT: 1-0
  TaggedPitchType  Usage%    AVG    OBP    SLG    OPS
1       Curveball    1.01  1.000  1.000  1.000  2.000
0        ChangeUp    9.09  0.500  0.556  1.000  1.556
7        Two-Seam    6.06  0.500  0.500  1.

In [3]:
df["PA_ID"] = (
    df["Date"].astype(str) + "_" +
    df["Pitcher"].astype(str) + "_" +
    df["Batter"].astype(str) + "_" +
    df["Inning"].astype(str) + "_" +
    df["PAofInning"].astype(str)
)

last_pitch_df = df.sort_values("PitchNo").groupby("PA_ID").tail(1)

print("\n\n PER PITCHER SUMMARIES ")

pitchers = df["Pitcher"].unique()

pitcher_report_rows = []

for pitcher in pitchers:
    df_p = df[df["Pitcher"] == pitcher].copy()
    df_p["Count"] = df_p["Balls"].astype(str) + "-" + df_p["Strikes"].astype(str)
    last_pitch_p = last_pitch_df[last_pitch_df["Pitcher"] == pitcher]

    if df_p.empty or last_pitch_p.empty:
        continue

    pitcher_report_rows.append([""])
    pitcher_report_rows.append([f"PITCHER: {pitcher}"])
    
    count_summary_p = df_p.groupby("Count").agg(
        PA=("PA", "sum"),
        AB=("AB", "sum"),
        H=("H", "sum"),
        BB=("BB", "sum"),
        HBP=("HBP", "sum"),
        SF=("SF", "sum"),
        _1B=("1B", "sum"),
        _2B=("2B", "sum"),
        _3B=("3B", "sum"),
        HR=("HR", "sum")
    ).reset_index()

    count_summary_p["AVG"] = count_summary_p["H"] / count_summary_p["AB"]
    count_summary_p["OBP"] = (count_summary_p["H"] + count_summary_p["BB"] + count_summary_p["HBP"]) / (
        count_summary_p["AB"] + count_summary_p["BB"] + count_summary_p["HBP"] + count_summary_p["SF"]
    )
    count_summary_p["TB"] = (
        count_summary_p["_1B"]
        + 2 * count_summary_p["_2B"]
        + 3 * count_summary_p["_3B"]
        + 4 * count_summary_p["HR"]
    )
    count_summary_p["SLG"] = count_summary_p["TB"] / count_summary_p["AB"]
    count_summary_p["OPS"] = count_summary_p["OBP"] + count_summary_p["SLG"]

    count_summary_p.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    count_summary_p.dropna(subset=["AVG", "OBP", "SLG"], inplace=True)
    count_summary_p[["AVG", "OBP", "SLG", "OPS"]] = count_summary_p[["AVG", "OBP", "SLG", "OPS"]].round(3)

    pitcher_report_rows.append(["Count", "PA", "AB", "AVG", "OBP", "SLG", "OPS"])
    for _, row in count_summary_p.sort_values(by="OPS", ascending=False).head(3).iterrows():
        pitcher_report_rows.append([
            row["Count"], row["PA"], row["AB"],
            row["AVG"], row["OBP"], row["SLG"], row["OPS"]
        ])

    best_counts_p = count_summary_p.sort_values(by="OPS", ascending=False).head(3)["Count"].tolist()

    for count in best_counts_p:
        df_wc_p = last_pitch_p[last_pitch_p["Count"] == count].copy()

        pitch_summary_p = df_wc_p.groupby("TaggedPitchType").agg(
            PA=("PlayResult", "count"),
            AB=("AB", "sum"),
            H=("H", "sum"),
            BB=("BB", "sum"),
            HBP=("HBP", "sum"),
            SF=("SF", "sum"),
            _1B=("1B", "sum"),
            _2B=("2B", "sum"),
            _3B=("3B", "sum"),
            HR=("HR", "sum")
        ).reset_index()

        total_pa_p = pitch_summary_p["PA"].sum()
        pitch_summary_p["Usage%"] = (pitch_summary_p["PA"] / total_pa_p) * 100

        pitch_summary_p["AVG"] = pitch_summary_p["H"] / pitch_summary_p["AB"]
        pitch_summary_p["OBP"] = (pitch_summary_p["H"] + pitch_summary_p["BB"] + pitch_summary_p["HBP"]) / (
            pitch_summary_p["AB"] + pitch_summary_p["BB"] + pitch_summary_p["HBP"] + pitch_summary_p["SF"]
        )
        pitch_summary_p["TB"] = (
            pitch_summary_p["_1B"]
            + 2 * pitch_summary_p["_2B"]
            + 3 * pitch_summary_p["_3B"]
            + 4 * pitch_summary_p["HR"]
        )
        pitch_summary_p["SLG"] = pitch_summary_p["TB"] / pitch_summary_p["AB"]
        pitch_summary_p["OPS"] = pitch_summary_p["OBP"] + pitch_summary_p["SLG"]

        pitch_summary_p.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
        pitch_summary_p.dropna(subset=["AVG", "OBP", "SLG"], inplace=True)

        pitch_summary_p["Usage%"] = pitch_summary_p["Usage%"].round(2)
        pitch_summary_p[["AVG", "OBP", "SLG", "OPS"]] = pitch_summary_p[["AVG", "OBP", "SLG", "OPS"]].round(3)

        pitcher_report_rows.append([""])
        pitcher_report_rows.append([f"PITCH TYPES IN WORST COUNT: {count}"])
        pitcher_report_rows.append(["TaggedPitchType", "Usage%", "AVG", "OBP", "SLG", "OPS"])
        for _, row in pitch_summary_p.sort_values(by="OPS", ascending=False).iterrows():
            pitcher_report_rows.append([
                row["TaggedPitchType"], row["Usage%"], row["AVG"],
                row["OBP"], row["SLG"], row["OPS"]
            ])

pitcher_report_df = pd.DataFrame(pitcher_report_rows)

with pd.ExcelWriter("FAL_COM_Report.xlsx", engine="openpyxl", mode="a") as writer:
    pitcher_report_df.to_excel(writer, sheet_name="Per Pitcher", index=False, header=False)



 PER PITCHER SUMMARIES 


In [4]:
fastball_types = ["Four-Seam", "Two-Seam", "Sinker", "Cutter", "Fastball"]

team_df = df[df["TaggedPitchType"].isin(fastball_types)].copy()
team_total = df.groupby("Count")["PitchNo"].count().reset_index(name="TotalPitches")
team_fb = team_df.groupby(["Count", "TaggedPitchType"]).size().reset_index(name="FB_Count")

team_fb_usage = team_fb.merge(team_total, on="Count")
team_fb_usage["Usage%"] = (team_fb_usage["FB_Count"] / team_fb_usage["TotalPitches"])

team_fb_pivot = team_fb_usage.pivot(index="Count", columns="TaggedPitchType", values="Usage%").fillna(0).reset_index()
team_fb_pivot.sort_values("Count", inplace=True)

team_fb_pivot["Fastball%"] = team_fb_pivot[fastball_types].sum(axis=1)

pitchers = df["Pitcher"].dropna().unique()
combined_pitchers_df = []

for pitcher in pitchers:
    df_p = df[df["Pitcher"] == pitcher]
    df_p_fb = df_p[df_p["TaggedPitchType"].isin(fastball_types)]

    if df_p_fb.empty:
        continue

    total_by_count = df_p.groupby("Count")["PitchNo"].count().reset_index(name="TotalPitches")
    fb_by_count = df_p_fb.groupby(["Count", "TaggedPitchType"]).size().reset_index(name="FB_Count")

    usage = fb_by_count.merge(total_by_count, on="Count")
    usage["Usage%"] = (usage["FB_Count"] / usage["TotalPitches"])

    pivot = usage.pivot(index="Count", columns="TaggedPitchType", values="Usage%").fillna(0).reset_index()
    pivot.sort_values("Count", inplace=True)

    pivot = pivot.merge(total_by_count, on="Count", how="left")

    cols = pivot.columns.tolist()
    if "TotalPitches" in cols:
        cols.insert(1, cols.pop(cols.index("TotalPitches")))
        pivot = pivot[cols]

    available_fastballs = [fb for fb in fastball_types if fb in pivot.columns]
    pivot["Fastball%"] = pivot[available_fastballs].sum(axis=1)

    pivot["Pitcher"] = pitcher

    combined_pitchers_df.append(pivot)

all_pitchers_df = pd.concat(combined_pitchers_df, ignore_index=True)
cols = ["Pitcher", "Count"] + [col for col in all_pitchers_df.columns if col not in ["Pitcher", "Count"]]
all_pitchers_df = all_pitchers_df[cols]

output_file = "FAL_COM_Report.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    team_fb_pivot.to_excel(writer, sheet_name="Team FB by Count", index=False)
    all_pitchers_df.to_excel(writer, sheet_name="Pitchers FB by Count", index=False)

wb = load_workbook(output_file)

for sheet_name in wb.sheetnames:
    if sheet_name.endswith("_FB_Count"):
        del wb[sheet_name]

def format_percent_columns(ws, header_row=1):
    percent_columns = []
    for col_idx, cell in enumerate(ws[header_row], start=1):
        if isinstance(cell.value, str) and "%" in cell.value:
            percent_columns.append(col_idx)

    for row in range(header_row + 1, ws.max_row + 1):
        for col_idx in percent_columns:
            col_letter = get_column_letter(col_idx)
            cell = ws[f"{col_letter}{row}"]
            if isinstance(cell.value, (int, float)):
                cell.number_format = '0.0%'

format_percent_columns(wb["Team FB by Count"])
format_percent_columns(wb["Pitchers FB by Count"])

wb.save(output_file)